In [7]:
import re
import csv

In [8]:
f = open("raw/school_report_raw.txt", "r")
text = f.read()
school_blocks = text.split("\n\n")

In [9]:
def process_one_school(school):
    assert(len(school) == 14)
    name_and_type = re.findall(r"(.+)(Public|Independent)", school[0])[0]
    name = name_and_type[0].strip()
    school_type = name_and_type[1]
    
    city_and_enrolment = re.findall(r"(.+)(Gr 12 enrolment: (\d+))", school[1])[0]
    city = city_and_enrolment[0].strip()
    enrolment = city_and_enrolment[1]
    
    esl, special_needs, french_imm = re.findall(r"(\d+\.?\d*|n\/a)+", school[2])
    average_exam_marks = re.findall(r"(\d+\.?\d*|n\/a)+", school[6])
    percent_exams_failed = re.findall(r"(\d+\.?\d*|n\/a)+", school[7])
    grad_rate = re.findall(r"(\d+\.?\d*|n\/a)+", school[11])
    delayed_rate = re.findall(r"(\d+\.?\d*|n\/a)+", school[12])
    
    overall_rating_group = re.findall(r"(Overall rating out of 10)? (\d+\.?\d*|n\/a)+", school[13])
    overall_rating = list(map(lambda rating_group: rating_group[1], overall_rating_group))
    
    records = []
    for i in range(2015, 2020):
        index = i - 2015
        records.append((name, school_type, i, city, enrolment, esl, special_needs, 
                       french_imm, average_exam_marks[index], percent_exams_failed[index],
                       grad_rate[index], delayed_rate[index], overall_rating[index]))
    
    return records

In [10]:
processed = []
failed = []

for school in school_blocks:
    try:
        records = process_one_school(school.split("\n"))
        processed = processed + records
    except:
        failed.append(school.split("\n")[0])

print(failed)
assert(len(failed) == 1) # Should be Kelowna Christian -- has no Gr 12 enrolment number

['Kelowna Christian Independent']


In [11]:
with open("processed_school_reports", "w") as out:
    csv_out = csv.writer(out)
    csv_out.writerow(["SCHOOL_NAME", "SCHOOL_TYPE", "YEAR", "LOCATION", "ENROLMENT", "PERCENT_ESL", "PERCENT_SPECIAL_NEEDS", "PERCENT_FRENCH_IMM",
                      "AVERAGE_EXAM_MARK", "PERCENT_EXAMS_FAILED", "PERCENT_GRAD", "PERCENT_DELAYED", "OVERALL"])
    for row in processed:
        csv_out.writerow(row)